# The JOIN operation

![rel](https://sqlzoo.net/w/images/c/ce/FootballERD.png)

## JOIN and UEFA EURO 2012

This tutorial introduces `JOIN` which allows you to use data from two or more tables. The tables contain all matches and goals from UEFA EURO 2012 Football Championship in Poland and Ukraine.

The data is available (mysql format) at <http://sqlzoo.net/euro2012.sql>

In [1]:
import os
import pandas as pd
import findspark
os.environ['SPARK_HOME'] =  '/opt/spark'
findspark.init()

from pyspark.sql import SparkSession
ss = (SparkSession.builder.appName('app00')
      .config('spark.sql.warehouse.dir', 'hdfs://quickstart.cloudera:8020/user/hive/warehouse')
      .config('hive.metastore.uris', 'thrift://quickstart.cloudera:9083')
      .enableHiveSupport().getOrCreate())

 ····


## 1.
The first example shows the goal scored by a player with the last name 'Bender'. The `*` says to list all the columns in the table - a shorter way of saying `matchid, teamid, player, gtime`

**Modify it to show the matchid and player name for all goals scored by Germany. To identify German players, check for: `teamid = 'GER'`**

In [2]:
game = pd.read_sql_table('game', engine)
goal = pd.read_sql_table('goal', engine)
eteam = pd.read_sql_table('eteam', engine)

In [3]:
goal.loc[goal['teamid']=='GER', ['matchid', 'player']]

,matchid,player
15,1008,Mario Gómez
21,1010,Mario Gómez
22,1010,Mario Gómez
27,1012,Lukas Podolski
29,1012,Lars Bender
61,1026,Philipp Lahm
63,1026,Sami Khedira
64,1026,Miroslav Klose
65,1026,Marco Reus
71,1030,Mesut Özil


## 2.

From the previous query you can see that Lars Bender's scored a goal in game 1012. Now we want to know what teams were playing in that match.

Notice in the that the column `matchid `in the `goal` table corresponds to the `id` column in the `game` table. We can look up information about game 1012 by finding that row in the `game` table.

**Show id, stadium, team1, team2 for just game 1012**

In [4]:
game.loc[game['id']==1012, ['id', 'stadium', 'team1', 'team2']]

,id,stadium,team1,team2
11,1012,Arena Lviv,DEN,GER


## 3.
You can combine the two steps into a single query with a JOIN.

```sql
SELECT *
  FROM game JOIN goal ON (id=matchid)
```

The **FROM** clause says to merge data from the goal table with that from the game table. The **ON** says how to figure out which rows in **game** go with which rows in **goal** - the **matchid** from **goal** must match **id** from **game**. (If we wanted to be more clear/specific we could say
`ON (game.id=goal.matchid)`

The code below shows the player (from the goal) and stadium name (from the game table) for every goal scored.

**Modify it to show the player, teamid, stadium and mdate for every German goal.**

In [5]:
(game.merge(goal, left_on='id', right_on='matchid')
    .loc[goal['teamid']=='GER', 
         ['player', 'teamid', 'stadium', 'mdate']])

,player,teamid,stadium,mdate
15,Mario Gómez,GER,Arena Lviv,9 June 2012
21,Mario Gómez,GER,Metalist Stadium,13 June 2012
22,Mario Gómez,GER,Metalist Stadium,13 June 2012
27,Lukas Podolski,GER,Arena Lviv,17 June 2012
29,Lars Bender,GER,Arena Lviv,17 June 2012
61,Philipp Lahm,GER,PGE Arena Gdansk,22 June 2012
63,Sami Khedira,GER,PGE Arena Gdansk,22 June 2012
64,Miroslav Klose,GER,PGE Arena Gdansk,22 June 2012
65,Marco Reus,GER,PGE Arena Gdansk,22 June 2012
71,Mesut Özil,GER,"National Stadium, Warsaw",28 June 2012


## 4.
Use the same `JOIN` as in the previous question.

**Show the team1, team2 and player for every goal scored by a player called Mario `player LIKE 'Mario%'`**

In [6]:
(game.merge(goal, left_on='id', right_on='matchid')
    .loc[goal['player'].str.startswith('Mario'),
         ['team1', 'team2', 'player']])

,team1,team2,player
15,GER,POR,Mario Gómez
21,NED,GER,Mario Gómez
22,NED,GER,Mario Gómez
32,IRL,CRO,Mario Mandžukic
35,IRL,CRO,Mario Mandžukic
37,ITA,CRO,Mario Mandžukic
44,ITA,IRL,Mario Balotelli
69,GER,ITA,Mario Balotelli
70,GER,ITA,Mario Balotelli


## 5.

The table `eteam` gives details of every national team including the coach. You can `JOIN` `goal` to `eteam` using the phrase goal `JOIN eteam on teamid=id`

**Show `player, teamid, coach, gtime` for all goals scored in the first 10 minutes `gtime<=10`**

In [7]:
(goal.merge(eteam, left_on='teamid', right_on='id')
    .loc[goal.gtime<=10, ['player', 'teamid', 'coach', 'gtime']])

,player,teamid,coach,gtime
7,Alan Dzagoev,RUS,Dick Advocaat,15
8,Roman Shirokov,RUS,Dick Advocaat,24
32,Silvestre Varela,POR,Paulo Bento,87
38,Antonio Di Natale,ITA,Cesare Prandelli,61


## 6.

To `JOIN` `game` with `eteam` you could use either
`game JOIN eteam ON (team1=eteam.id)` or `game JOIN eteam ON (team2=eteam.id)`

Notice that because `id` is a column name in both `game` and `eteam` you must specify `eteam.id` instead of just `id`

**List the the dates of the matches and the name of the team in which 'Fernando Santos' was the team1 coach.**

In [8]:
a = game.merge(eteam, left_on='team1', right_on='id')
a.loc[a['coach']=='Fernando Santos', ['mdate', 'teamname']]

,mdate,teamname
3,12 June 2012,Greece
4,16 June 2012,Greece


## 7.

**List the player for every goal scored in a game where the stadium was 'National Stadium, Warsaw'**

In [9]:
a = goal.merge(game, left_on='matchid', right_on='id')
a.loc[a['stadium']=='National Stadium, Warsaw', ['player']].drop_duplicates()

,player
0,Robert Lewandowski
1,Dimitris Salpingidis
10,Alan Dzagoev
11,Jakub Blaszczykowski
13,Giorgos Karagounis
60,Cristiano Ronaldo
69,Mario Balotelli
71,Mesut Özil


## 8. More difficult questions

The example query shows all goals scored in the Germany-Greece quarterfinal.
**Instead show the name of all players who scored a goal against Germany.**

> __HINT__   
> Select goals scored only by non-German players in matches where GER was the id of either **team1** or **team2**.
> You can use `teamid!='GER'` to prevent listing German players.
> You can use `DISTINCT` to stop players being listed twice.

In [10]:
a = game.merge(goal, left_on='id', right_on='matchid')
a.loc[((a['team1']=='GER') | (a['team2']=='GER')) & (a['teamid']!='GER'), 
      ['player']].drop_duplicates()

,player
23,Robin van Persie
28,Michael Krohn-Dehli
62,Georgios Samaras
66,Dimitris Salpingidis
69,Mario Balotelli


## 9.
Show teamname and the total number of goals scored.

> __COUNT and GROUP BY__  
> You should COUNT(*) in the SELECT line and GROUP BY teamname

In [11]:
a = eteam.merge(goal, left_on='id', right_on='teamid')
a[['teamname', 'player']].groupby('teamname').count().reset_index()

,teamname,player
0,Croatia,4
1,Czech Republic,4
2,Denmark,4
3,England,5
4,France,3
5,Germany,10
6,Greece,5
7,Italy,6
8,Netherlands,2
9,Poland,2


## 10.

**Show the stadium and the number of goals scored in each stadium.**

In [12]:
a = game.merge(goal, left_on='id', right_on='matchid')
a[['stadium', 'player']].groupby('stadium').count().reset_index()

,stadium,player
0,Arena Lviv,9
1,Donbass Arena,7
2,Metalist Stadium,7
3,"National Stadium, Warsaw",9
4,Olimpiyskiy National Sports Complex,14
5,PGE Arena Gdansk,13
6,Stadion Miejski (Poznan),8
7,Stadion Miejski (Wroclaw),9


## 11.
**For every match involving 'POL', show the matchid, date and the number of goals scored.**

In [13]:
a = game.merge(goal, left_on='id', right_on='matchid')
a.loc[(a['team1']=='POL') | (a['team2']=='POL'), 
  ['matchid', 'mdate', 'player']].groupby(['matchid', 'mdate']).count().reset_index()

,matchid,mdate,player
0,1001,8 June 2012,2
1,1004,12 June 2012,2
2,1005,16 June 2012,1


## 12.
**For every match where 'GER' scored, show matchid, match date and the number of goals scored by 'GER'**

In [14]:
a = game.merge(goal, left_on='id', right_on='matchid')
a.loc[a['teamid']=='GER', 
      ['matchid', 'mdate', 'player']].groupby(['matchid', 'mdate']).count().reset_index()

,matchid,mdate,player
0,1008,9 June 2012,1
1,1010,13 June 2012,2
2,1012,17 June 2012,2
3,1026,22 June 2012,4
4,1030,28 June 2012,1


## 13.
List every match with the goals scored by each team as shown. This will use "CASE WHEN" which has not been explained in any previous exercises.

no  | mdate      | team1  | score1 | team2 | score2
----|------------|--------|--------|-------|--------
1   | July 2012  | ESP    | 4      | ITA   | 0
10  | June 2012  | ESP    | 1      | ITA   | 1
10  | June 2012  | IRL    | 1      | CRO   | 3
... | ...        | ...    | ...    | ...   | ...

Notice in the query given every goal is listed. If it was a team1 goal then a 1 appears in score1, otherwise there is a 0. You could SUM this column to get a count of the goals scored by team1. **Sort your result by mdate, matchid, team1 and team2.**

In [15]:
s = (goal[['matchid', 'teamid', 'player']]
     .groupby(['matchid', 'teamid'])
     .count().reset_index())

(game.merge(s, how='left', left_on=['id', 'team1'], right_on=['matchid', 'teamid'])
    .merge(s, how='left', left_on=['id', 'team2'], right_on=['matchid', 'teamid'])
    .loc[:, ['mdate', 'team1', 'player_x', 'team2', 'player_y']]
    .fillna(0)
    .rename(columns={'player_x': 'score1', 'player_y': 'score2'})
    .sort_values('mdate'))

,mdate,team1,score1,team2,score2
30,1 July 2012,ESP,4.0,ITA,0.0
13,10 June 2012,IRL,1.0,CRO,3.0
12,10 June 2012,ESP,1.0,ITA,1.0
19,11 June 2012,UKR,2.0,SWE,1.0
18,11 June 2012,FRA,1.0,ENG,1.0
2,12 June 2012,GRE,1.0,CZE,2.0
3,12 June 2012,POL,1.0,RUS,1.0
8,13 June 2012,DEN,2.0,POR,3.0
9,13 June 2012,NED,1.0,GER,2.0
14,14 June 2012,ITA,1.0,CRO,1.0
